In [1]:
from collections import Counter


                                    Directory structure:
Initial files:
    - lab2_crypt.ipynb - current notebook with second lab
    - neznaika.txt - user text to count Index of coincidence for russian language
    - variant13_raw.txt - ciphered text that we have to dechipher of variant 13
Files/dirs created during working with curren`t notebook:
    - dir encoded:
        -- ../'vigener_res{}.txt'.format(key_len) - encoded text from 'neznaika.txt' with key of length key_len from dictionary key_dict
    - variant13.txt - a little preprocessing of 'variant13_raw.txt', getting rid of '\n' symbol
    - 'vigener_check_decode{}.txt'.format(key_len) - decoded text with key of length key_len 

Preprocessing is essential to make rid of punctuation, uppercase and spaces. We also change 'ё' to 'е'

In [2]:
def preprocess(path):
    '''Leaves only lowercase russian chars without spaces'''
    newpath='lab2_prep.txt'
    alph='абвгдеёжзийклмнопрстуфхцчшщъыьэюя'
    with open(path, 'r') as infile, \
     open(newpath, 'w') as outfile:
        data = infile.read()
        data = data.lower()
        data = data.replace('ё', 'e')
        result=''
        for char in data:
            if char in alph:
                result+=char
        #sanity check (may be less than 32 if some rare symbols are missing)
        assert len(set(result)) <= 32 
        outfile.write(result)

In [3]:
preprocess('neznaika.txt')

Constants for lab

In [4]:
#dictionary of format "len of a key: key"
key_dict = {
    2: 'он',
    3: 'сок',
    4: 'вино',
    5: 'каток',
    17: 'будькакдомапутник',
}

In [5]:
alph='абвгдежзийклмнопрстуфхцчшщъыьэюя'

Vigenère cipher

In [6]:
def vigener(path, key_len):
    '''Creates encoded text from plain text using key from key_dict with len key_len'''
    key = key_dict[key_len]
    with open(path, 'r') as infile, \
    open('encoded/vigener_res{}.txt'.format(key_len), 'w') as outfile:
        data = infile.read()
        result=''
        for i, char in enumerate(data):
            char = alph[(alph.find(char) + alph.find(key[i % len(key)])) % len(alph)]
            result+=char
        outfile.write(result)

In [7]:
vigener('lab2_prep.txt', 4)

Decoding function to make sure everything is going OK

In [8]:
def decode_vigener(path, key, print_text=False):
    '''Decode vigener !WITH KNOWN KEY!'''
    with open(path, 'r') as infile, \
    open('vigener_check_decode{}.txt'.format(len(key)), 'w') as outfile:
        data = infile.read()
        result=''
        for i, char in enumerate(data):
            char =  alph[alph.find(char) - alph.find(key[i % len(key)])]
            result += char
        outfile.write(result)
    if print_text:
        print(result)

In [9]:
decode_vigener('encoded/vigener_res4.txt', 'вино')

Applying vigener using all keys we have (results are in dir 'encoded')

In [10]:
for key_len in key_dict.keys():
    vigener('lab2_prep.txt', key_len)

Counting Index of coincidence (effective for small r (2 <= r <= 5)). For bigger r we prefer to use symbol coincidence statistics

In [11]:
def coincidence_count(path):
    with open(path, 'r') as infile:
        data = infile.read()
        n = len(data)
        counter = Counter(data) #count number of occurences of each letter in text
    index = sum([counter[i]* (counter[i]-1) for i in counter.keys()]) / (n * (n-1))
    return index 
    

Counting coincidence index of user text

In [12]:
coincidence_count('neznaika.txt')

0.058285657003133866

Counting coincidence index of user text, encoded with different keys

In [13]:
for key_len in key_dict.keys():
    result = coincidence_count('encoded/vigener_res{}.txt'.format(key_len))
    print('Length of key: {}, coincidence index: {}'.format(key_len, result))

Length of key: 2, coincidence index: 0.04535229303713382
Length of key: 3, coincidence index: 0.04030504957234696
Length of key: 4, coincidence index: 0.03707842237387874
Length of key: 5, coincidence index: 0.03790313145538461
Length of key: 17, coincidence index: 0.03365344974743112


Starting working with raw file (but we have to get rid of '\n' symbol at first)

In [14]:
with open('variant13_raw.txt', 'r') as infile, \
     open('variant13.txt', 'w') as outfile:
        data = infile.read()
        data = data.replace('\n', '')
        print(Counter(data))
        outfile.write(data)

Counter({'н': 270, 'х': 252, 'т': 243, 'у': 237, 'р': 235, 'ц': 217, 'п': 208, 'м': 187, 'ш': 184, 'щ': 182, 'ы': 171, 'о': 169, 'к': 162, 'и': 162, 'й': 160, 'ъ': 160, 'а': 157, 'с': 154, 'ч': 152, 'ь': 151, 'ф': 151, 'л': 150, 'ю': 141, 'д': 137, 'е': 136, 'я': 135, 'з': 132, 'б': 122, 'э': 117, 'в': 110, 'г': 109, 'ж': 98})


As we see, r seems to be much larger then 5. Or even possibly larger than 17(?). So Coincidence count seems to be not the most efficient way here.

In [15]:
coincidence_count('variant13.txt')

0.033177901938147646

Symbol_statistics shows how often letters are the same on distance r. The more they are, the bigger the chance that we have found the real r.

In [16]:
def symbol_statistics(path, r):
    with open(path, 'r') as infile:
        data = infile.read()
        n = len(data)
        result = 0
        for i in range (n-r):
            if data[i] == data[i+r]:
                result += 1
    return result

In [17]:
for r in range (6, 31):
    result = symbol_statistics('variant13.txt', r)
    print('Length of key(r): {}, symbol statistics(coincidence): {}'.format(r, result))

Length of key(r): 6, symbol statistics(coincidence): 158
Length of key(r): 7, symbol statistics(coincidence): 150
Length of key(r): 8, symbol statistics(coincidence): 183
Length of key(r): 9, symbol statistics(coincidence): 182
Length of key(r): 10, symbol statistics(coincidence): 195
Length of key(r): 11, symbol statistics(coincidence): 170
Length of key(r): 12, symbol statistics(coincidence): 197
Length of key(r): 13, symbol statistics(coincidence): 181
Length of key(r): 14, symbol statistics(coincidence): 154
Length of key(r): 15, symbol statistics(coincidence): 182
Length of key(r): 16, symbol statistics(coincidence): 155
Length of key(r): 17, symbol statistics(coincidence): 288
Length of key(r): 18, symbol statistics(coincidence): 151
Length of key(r): 19, symbol statistics(coincidence): 158
Length of key(r): 20, symbol statistics(coincidence): 171
Length of key(r): 21, symbol statistics(coincidence): 156
Length of key(r): 22, symbol statistics(coincidence): 189
Length of key(r): 

It seems that r = 17 shows the result we have expected. The symbol statistics of r=17 is >> than other symbol statistics. To check this, lets check some random numbers >30 and several numbers that divide by 17 without remainders

In [18]:
for r in [17, 17*2, 17*3, 17*4]:
    result = symbol_statistics('variant13.txt', r)
    print('Length of key(r): {}, symbol statistics(coincidence): {}'.format(r, result))

Length of key(r): 17, symbol statistics(coincidence): 288
Length of key(r): 34, symbol statistics(coincidence): 341
Length of key(r): 51, symbol statistics(coincidence): 340
Length of key(r): 68, symbol statistics(coincidence): 324


In [19]:
for r in [17*2+6, 17*3+3, 17*3+15]:
    result = symbol_statistics('variant13.txt', r)
    print('Length of key(r): {}, symbol statistics(coincidence): {}'.format(r, result))

Length of key(r): 40, symbol statistics(coincidence): 150
Length of key(r): 54, symbol statistics(coincidence): 175
Length of key(r): 66, symbol statistics(coincidence): 161


Well, it works. For r =17 results are much higher.

Counting the frequency of each letter in user text to understand the possible frequency of words in ciphertext

In [20]:
with open('lab2_prep.txt', 'r') as infile:
        data = infile.read()
        russian_freq = Counter(data)
russian_freq = sorted(russian_freq.items(), key=lambda item: (-item[1], item[0]))
russian_freq

[('о', 737),
 ('а', 551),
 ('е', 487),
 ('и', 480),
 ('н', 412),
 ('л', 410),
 ('т', 374),
 ('с', 328),
 ('к', 323),
 ('р', 248),
 ('в', 230),
 ('м', 225),
 ('у', 200),
 ('ы', 190),
 ('д', 168),
 ('з', 136),
 ('б', 126),
 ('п', 126),
 ('ь', 119),
 ('г', 109),
 ('ч', 105),
 ('й', 101),
 ('ш', 95),
 ('я', 94),
 ('ж', 59),
 ('х', 48),
 ('ц', 43),
 ('ю', 37),
 ('э', 24),
 ('щ', 13),
 ('ф', 1)]

We will try to find key of length 17 using statistics russian_freq

In [21]:
def key_finder(path, r, debug = False, debug_var = 2):
    '''Finds the most possible key of given length r.
       If debug == True, looks for the second most frequent value in each block and, 
       if the difference between most frequent value and second most frequent value is <= debug_var, 
       considers second value as the most possible for now'''
    with open(path, 'r') as infile:
        data = infile.read()
        n = len(data)
        key = ''
        if debug:
            #we have r blocks
            for block_num in range(r):
                #in the r-th block we have every r-th symbol starting from number of block
                text = [data[i] for i in range(block_num, n, r )] 
                block_freq = Counter(text)
                #showing frequencies of symbols in the block
                block_freq = sorted(block_freq.items(), key=lambda item: (-item[1], item[0]))
                #if frequency of the most frequent letter in block is just SLIGHTLY higher 
                #than frequency of second most freq letter
                if ((block_freq[0][1] - block_freq[1][1]) <= debug_var):
                    key += alph[abs((alph.find(block_freq[1][0]) - alph.find(russian_freq[0][0])) % len(alph))]
                else:
                    key += alph[abs((alph.find(block_freq[0][0]) - alph.find(russian_freq[0][0])) % len(alph))]
        else:
            for block_num in range(r):
                text = [data[i] for i in range(block_num, n, r )] 
                block_freq = Counter(text)
                print(block_freq)
                block_freq = sorted(block_freq.items(), key=lambda item: (-item[1], item[0]))
                key += alph[abs((alph.find(block_freq[0][0]) - alph.find(russian_freq[0][0])) % len(alph))]
    return key

In [22]:
key_finder('variant13.txt', 17)

Counter({'ю': 39, 'х': 22, 'р': 20, 'б': 19, 'в': 19, 'э': 17, 'ш': 16, 'ы': 16, 'а': 14, 'г': 13, 'ъ': 13, 'ь': 12, 'ф': 12, 'л': 10, 'т': 9, 'п': 8, 'м': 8, 'я': 8, 'у': 7, 'с': 6, 'ц': 4, 'з': 4, 'е': 4, 'и': 4, 'ч': 3, 'о': 3, 'й': 2, 'ж': 2, 'н': 1})
Counter({'у': 36, 'ь': 30, 'ы': 21, 'ц': 21, 'о': 21, 'щ': 20, 'р': 18, 'а': 17, 'ш': 14, 'ю': 13, 'э': 11, 'ъ': 9, 'я': 9, 'т': 9, 'н': 8, 'к': 8, 'п': 7, 'б': 6, 'ф': 5, 'е': 5, 'ж': 5, 'й': 4, 'с': 4, 'х': 4, 'з': 3, 'л': 2, 'м': 2, 'в': 2, 'г': 1})
Counter({'й': 32, 'т': 30, 'с': 26, 'д': 25, 'х': 19, 'ф': 18, 'ц': 17, 'п': 15, 'ж': 15, 'м': 14, 'ч': 14, 'о': 12, 'н': 9, 'г': 8, 'и': 7, 'у': 7, 'е': 7, 'р': 6, 'я': 5, 'з': 5, 'л': 4, 'щ': 4, 'а': 4, 'б': 3, 'э': 2, 'ы': 2, 'к': 2, 'в': 2, 'ь': 1})
Counter({'ц': 39, 'н': 28, 'х': 26, 'у': 25, 'и': 19, 'к': 18, 'щ': 16, 'р': 15, 'ш': 15, 'ъ': 14, 'т': 13, 'ы': 12, 'м': 11, 'ф': 10, 'с': 8, 'ч': 7, 'з': 5, 'я': 5, 'п': 5, 'о': 4, 'д': 4, 'л': 3, 'э': 3, 'й': 3, 'е': 3, 'г': 2, 'б': 1

'реыинтуезразлъчяя'

Well, the key is not readable yet so lets look closer at the blocks where freqs of first and secomnd symbols are almost the same

In [23]:
key_finder('variant13.txt', 17, debug = True, debug_var = 2)

'рединабезраюличия'

Two possible keys that we have are:
    1. 'реыинтуезразлъчяя'
    2. 'рединабезраюличия'
We can conclude that result is something like 'рединабезразличия'. 
Lets dechifer text with this key and see if we are right

In [24]:
decode_vigener('variant13.txt',  'рединабезразличия', print_text = True)

эускаваторприземисыыйидлинныйсловноыепловозсдалековыцесеннойсуставчатчйтягойичудовищныхзубатымковшомгусоницыглубоковминафисьвпочвуоставляидвенепрерывныерекристыедорожкиразищеесоляройлязгаювееоноперлонеразбсраядорогииготовокылосокрушитьвсенйсвоемпутионочудивегенералприроскмостуневсилахпошеволитьсяеслиэтоконырольныйсюрпризтолесемирочченьвысоуогообудущемведьмйкемненияапотомстщахизамешательствчнеожиданносхлынуфиосталосьтолькосшокойствиеиглубокйяуверенностьразухведьмакапустьдажоиначинающеговсерйвногибчеибыстрееыупыхинстинктовдиуоймашиныпобедитькесхитростнуюмощьхожноибезоружияодцойлишьсилоймыслиослизнаешькакгенещалзналпокатольколтеорииноведьвтомссостоитсмыслконтщольныхполевыхзадйнийвпривязкетеоротическихзнанийкроальнойобстановкечдновременномелькцулашальнаяивданндймоментмалоуместцаямыслишкавотзачомустроилииспытансевпустоминенаселонномпаркетакойэкъкаваторнагородсксхулицахстолькобылсегопорушилзадеситьлетнеотрослобыстакимеетсякарьерцыйгусеничныйэкскйватормоделимоделсачертегознаеткакчймоделимного

In [25]:
alph='абвгдежзийклмнопрстуфхцчшщъыьэюя'

In [26]:
decode_vigener('variant13.txt',  'родинабезразличия', print_text = True)

экскаваторприземистыйидлинныйсловнотепловозсдалековынесеннойсуставчатойтягойичудовищнымзубатымковшомгусеницыглубоковминалисьвпочвуоставляядвенепрерывныеребристыедорожкиразящеесоляройлязгающееоноперлонеразбираядорогииготовобылосокрушитьвсенасвоемпутионочудищегенералприроскместуневсилахпошевелитьсяеслиэтоконтрольныйсюрпризтовесемирочченьвысокогообудущемведьмакемненияапотомстрахизамешательствонеожиданносхлынулиосталосьтолькоспокойствиеиглубокаяуверенностьразумведьмакапустьдажеиначинающеговсеравногибчеибыстреетупыхинстинктовдикоймашиныпобедитьбесхитростнуюмощьможноибезоружияоднойлишьсилоймыслиеслизнаешькакгенералзналпокатольковтеорииноведьвтомисостоитсмыслконтрольныхполевыхзаданийвпривязкетеоретическихзнанийкреальнойобстановкеодновременномелькнулашальнаяивданныймоментмалоуместнаямыслишкавотзачемустроилииспытаниевпустоминенаселенномпаркетакойэкскаваторнагородскихулицахстолькобывсегопорушилзадесятьлетнеотрослобыитакимеетсякарьерныйгусеничныйэкскаватормоделимоделиачертегознаеткакоймоделимного

Having played with text a little, we found out that the right key is 'родинабезразличия' instead of 'рединабезразличия'. So we managed to decode the text fully.